This notebook contains implementation details of the four inital baselines model which were implementated using TensorFlow framework. The models are Vanilla VQA, TDA, MFB, and MFH

# Libraries

In [ ]:
import json
import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Flatten, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model
from collections import Counter
import matplotlib.font_manager as fm
import matplotlib as mpl
from matplotlib.legend import FontProperties
import multidict as multidict
import numpy as np
from keras.utils import to_categorical
import os
import re
from PIL import Image
from os import path
from wordcloud import WordCloud
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Embedding, LSTM, Bidirectional, concatenate
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.layers import multiply
from tensorflow.keras.layers import Concatenate, Dense, Input, LSTM, Embedding, Dropout, Activation,Flatten,Reshape,dot
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import GRU
from tensorflow.keras.callbacks import Callback, EarlyStopping,ModelCheckpoint,TensorBoard
from transformers import BertTokenizer
from transformers import TFBertModel
from tensorflow.keras.layers import Layer, MultiHeadAttention
import cv2


dataset_path = "<path of the dataset>"
image_path = "<path of the images>"
models_path = "<saved models path>"

In [ ]:
# pip install tensorflow==2.11.0

# Read Excel Files

In [ ]:
train = pd.read_excel(dataset_path+"train_bvqa.xlsx")
valid = pd.read_excel(dataset_path+"valid_bvqa.xlsx")
test = pd.read_excel(dataset_path+"test_bvqa.xlsx")

print(train.shape)
print(valid.shape)
print(test.shape)

(9557, 4)
(720, 4)
(774, 4)


In [ ]:
len(test['class'].unique())

140

# Mapping

In [ ]:
ans = test['answers'].tolist()
label = test['class'].tolist()
a = zip(ans,label)
# Remove duplicates by converting to a set, then back to a list
unique_data = list(set(a))
# Sort the unique data based on the second item in each tuple
sorted_unique_data = sorted(unique_data, key=lambda x: x[1])

labels = list()
for i in sorted_unique_data:
  labels.append(i[0])

def label_map(row):
    return labels[row]

In [ ]:
yes_no = ['হ্যাঁ','না']
count = ['কয়েকজন','দুইজন','একজন', 'দুই','অনেকগুলো','তিনজন','চারজন','তিন','অনেকজন','কয়েকটি','পাঁচ','৩',
 'অনেক','পাঁচজন','একটি','চার','দুইটি','৪','ছয়জন','ছয়','২','এক','তিনটি','১','সাত','৫','শূন্য','সাতজন','দুইটা',
         'দুজন','একটা','নয়','৬','আট']

def category_map(row):
    if row in yes_no:
        return 'yes/no'
    elif row in count:
        return 'counting'
    else:
      return 'other'

In [ ]:
train['category'] = train['answers'].apply(category_map)
valid['category'] = valid['answers'].apply(category_map)
test['category'] = test['answers'].apply(category_map)

In [ ]:
train['category'].value_counts()

category
other       4701
counting    3900
yes/no       956
Name: count, dtype: int64

In [ ]:
valid['category'].value_counts()

category
other       355
counting    293
yes/no       72
Name: count, dtype: int64

In [ ]:
test['category'].value_counts()

category
other       384
counting    313
yes/no       77
Name: count, dtype: int64

# Make Pickle Files

In [ ]:
# Path to your image directory (assuming you have image files corresponding to 'image_name' in the dataset)
image_directory = image_path

# Define the image size expected by ResNet50
image_names = train['filename']
image_size = (224, 224)
images = []
#Load and preprocess images
for i, image_name in enumerate(image_names):
     image_path = image_directory + image_name  # Create the full path to the image
     img = load_img(image_path, target_size=image_size)  # Load and resize the image
     print(i)
     img = img_to_array(img)  # Convert to NumPy array
     img = preprocess_input(img)  # Apply ResNet's preprocessing
     images.append(img)
# Convert the list of images to a NumPy array
images = np.array(images)

with open(dataset_path+'train_images_allclass.pkl','wb') as f:
      pkl.dump(images, f)

In [ ]:
# Define the image size expected by ResNet50
image_names = valid['filename']
image_size = (224, 224)
images = []
#Load and preprocess images
for i, image_name in enumerate(image_names):
     image_path = image_directory + image_name  # Create the full path to the image
     img = load_img(image_path, target_size=image_size)  # Load and resize the image
     print(i)
     img = img_to_array(img)  # Convert to NumPy array
     img = preprocess_input(img)  # Apply ResNet's preprocessing
     images.append(img)
# Convert the list of images to a NumPy array
images = np.array(images)

with open(dataset_path +'valid_images_allclass.pkl','wb') as f:
      pkl.dump(images, f)

In [ ]:
# Define the image size expected by ResNet50
image_names = test['filename']
image_size = (224, 224)
images = []
#Load and preprocess images
for i, image_name in enumerate(image_names):
     image_path = image_directory + image_name  # Create the full path to the image
     img = load_img(image_path, target_size=image_size)  # Load and resize the image
     print(i)
     img = img_to_array(img)  # Convert to NumPy array
     img = preprocess_input(img)  # Apply ResNet's preprocessing
     images.append(img)
# Convert the list of images to a NumPy array
images = np.array(images)

with open(dataset_path +'test_images_allclass.pkl','wb') as f:
      pkl.dump(images, f)

# Load the Images

In [ ]:
# Load a Pickle file

# train images
with open(dataset_path +"train_images_allclass.pkl", 'rb') as file:
    train_images = pkl.load(file)
print(train_images.shape)

# valid images
with open(dataset_path +"valid_images_allclass.pkl", 'rb') as file:
    valid_images = pkl.load(file)
print(valid_images.shape)

# test images
with open(dataset_path +"test_images_allclass.pkl", 'rb') as file:
    test_images = pkl.load(file)
print(test_images.shape)


(9557, 224, 224, 3)
(720, 224, 224, 3)
(774, 224, 224, 3)


# Encoding question

In [ ]:
#Preprocess the text data (questions)
questions_input = train['questions']
basic_tokenizer = Tokenizer()
basic_tokenizer.fit_on_texts(train['questions'])
vocab_size = len(basic_tokenizer.word_index) + 1
print("Vocabulary Size:", vocab_size)

# train questions
train_questions = basic_tokenizer.texts_to_sequences(train['questions'])
train_seq = pad_sequences(train_questions, maxlen=15,padding='post')
# valid questions
valid_questions = basic_tokenizer.texts_to_sequences(valid['questions'])
valid_seq = pad_sequences(valid_questions, maxlen=15,padding='post')
# test questions
test_questions = basic_tokenizer.texts_to_sequences(test['questions'])
test_seq = pad_sequences(test_questions, maxlen=15,padding='post')

print("Train: ",train_seq.shape)
print("Valid: ",valid_seq.shape)
print("Test: ",test_seq.shape)


Vocabulary Size: 1499
Train:  (9557, 15)
Valid:  (720, 15)
Test:  (774, 15)


In [ ]:
train_questions[1]

array([117, 603,  12,   7,   3,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0], dtype=int32)

In [ ]:
train_questions.shape

(5841, 15)

# Callbacks

In [ ]:
# Saved the Best Model
filepath = models_path+ '<model_name>.h5'

callback = keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=5)

checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=2, save_best_only=True,
                                             save_weights_only=True, mode='max')

callback_list = [callback,checkpoint]
num_of_classes = 140

# Evalution Metrics

In [ ]:
def print_metrices(true,pred):
    print("Accuracy : ",round(accuracy_score(true,pred),4))
    print("Precison : ",round(precision_score(true,pred, average = 'weighted'),4))
    print("Recall : ",round(recall_score(true,pred,  average = 'weighted'),4))
    print("F1 : ",round(f1_score(true,pred,  average = 'weighted'),4))

# MFB Layer (2017)

In [ ]:
from keras.layers import Layer, Concatenate, GlobalAveragePooling2D
import tensorflow as tf
class MFB(Layer):
    def __init__(self, latent_dim, dropout_rate=0.5, pool_size=2, **kwargs):
        self.latent_dim = latent_dim
        self.dropout_rate = dropout_rate
        self.pool_size = pool_size
        super(MFB, self).__init__(**kwargs)

    def build(self, input_shape):
        self.Ui = self.add_weight(name='Ui', shape=(input_shape[0][-1], self.latent_dim),
                                   initializer='glorot_uniform', trainable=True)
        self.Vi = self.add_weight(name='Vi', shape=(input_shape[1][-1], self.latent_dim),
                                   initializer='glorot_uniform', trainable=True)
        super(MFB, self).build(input_shape)

    def call(self, inputs):
        x, y = inputs

        # Factorized matrices
        Ui = self.Ui
        Vi = self.Vi

        # Element-wise multiplication
        mfb = tf.matmul(x, Ui) * tf.matmul(y, Vi)

        # Sum pooling
        mfb = tf.nn.avg_pool1d(tf.expand_dims(mfb, axis=2), ksize=self.pool_size, strides=self.pool_size, padding='VALID')
        mfb = tf.squeeze(mfb, axis=2)

        # Dropout layer
        mfb = Dropout(self.dropout_rate)(mfb)

        # Power normalization
        mfb = tf.sign(mfb) * tf.sqrt(tf.abs(mfb) + 1e-6)

        # L2 normalization
        mfb = tf.nn.l2_normalize(mfb, axis=-1)

        return mfb

    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], self.latent_dim)

In [ ]:
latent_dim = 128  # Define latent dimensionality
dropout_rate = 0.2  # Define dropout rate
pool_size = 2  # Define pool size for sum pooling


#ResNet101
base_model = tf.keras.applications.ResNet152(weights='imagenet', include_top=False,input_shape=(224, 224, 3))
base_model.trainable = False
y = base_model.output
conv_layer = base_model.output
pool = GlobalAveragePooling2D()(conv_layer)
visual_features = pool
# print(visual_features.shape)
#GRU
sequence_input = Input(shape=(15,), dtype="int32")
embedded_sequences = Embedding(10000, 300)(sequence_input)
# Getting our GRU outputs
lstm_layer1 = LSTM(1024,return_sequences=True)(embedded_sequences)
last_output1 = lstm_layer1[:, -1, :]

# Getting our GRU outputs
lstm_layer2 = LSTM(1024)(lstm_layer1)
textual_features = Concatenate(axis=-1)([last_output1, lstm_layer2])
# print(textual_features.shape)
# MFB layer
mfb_output = MFB(latent_dim, dropout_rate, pool_size)([visual_features, textual_features])

output = Dense(num_of_classes, activation="softmax")(mfb_output)

# Create model
model = Model(inputs=[base_model.input, sequence_input], outputs=output )


In [ ]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit([train_images, train_seq], train['class'],
          epochs=25,batch_size=64,
          validation_data=([valid_images, valid_seq], valid['class']),callbacks=callback_list)

In [ ]:
# Evaluate the model on the test set
model.load_weights(models_path+'mfb_allclass.h5')
test_loss, test_accuracy = model.evaluate([test_images, test_seq], test['class'])
print(f'Test Loss: {round(test_loss,3)}, Test Accuracy: {round(test_accuracy,4)}')

25/25 [==============================] - 69s 3s/step - loss: 1.7743 - accuracy: 0.5891
Test Loss: 1.774, Test Accuracy: 0.5891


In [ ]:
# prediction
y_pred=model.predict([test_images, test_seq])
y_pred= np.argmax(y_pred, axis=-1)
# print matrices
print_metrices(test['class'],y_pred)

25/25 [==============================] - 64s 3s/step
Accuracy :  0.5891
Precison :  0.5183
Recall :  0.5891
F1 :  0.5421


In [ ]:
test['mfb'] = y_pred
test['mfb'] = test['mfb'].apply(label_map)
test

In [ ]:
yes_no = test[test['category']=='yes/no']
counting = test[test['category']=='counting']
other = test[test['category']=='other']

In [ ]:
# Generate the classification report
print_metrices(counting['answers'], counting['mfb'])

Accuracy :  0.4888
Precison :  0.4496
Recall :  0.4888
F1 :  0.4591


# MFH Layer (2019)

In [ ]:
from keras.layers import Layer, Concatenate, GlobalAveragePooling2D
import tensorflow as tf

class MFH(Layer):
    def __init__(self, latent_dim, num_blocks=1, dropout_rate=0.5, pool_size=2, **kwargs):
        self.latent_dim = latent_dim
        self.num_blocks = num_blocks
        self.dropout_rate = dropout_rate
        self.pool_size = pool_size
        self.mfb_blocks = [MFB(latent_dim, dropout_rate, pool_size) for _ in range(num_blocks)]
        super(MFH, self).__init__(**kwargs)

    def call(self, inputs):
        mfb_outputs = [mfb_block(inputs) for mfb_block in self.mfb_blocks]
        concatenated_output = Concatenate(axis=-1)(mfb_outputs)
        return concatenated_output

    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], self.latent_dim * self.num_blocks)

In [ ]:
latent_dim = 128  # Define latent dimensionality
num_blocks = 3  # Define number of MFB blocks
dropout_rate = 0.1  # Define dropout rate
pool_size = 2  # Define pool size for sum pooling

#ResNet101
base_model = tf.keras.applications.ResNet152(weights='imagenet', include_top=False,input_shape=(224, 224, 3))
base_model.trainable = False
y = base_model.output
conv_layer = base_model.output
pool = GlobalAveragePooling2D()(conv_layer)
visual_features = pool

#GRU
sequence_input = Input(shape=(15,), dtype="int32")
embedded_sequences = Embedding(10000, 300)(sequence_input)
# Getting our GRU outputs
lstm_layer1 = LSTM(1024,return_sequences=True)(embedded_sequences)
last_output1 = lstm_layer1[:, -1, :]

# Getting our GRU outputs
lstm_layer2 = LSTM(1024)(lstm_layer1)
textual_features = Concatenate(axis=-1)([last_output1, lstm_layer2])
# print(textual_features.shape)
# MFH layer
mfh_output = MFH(latent_dim, num_blocks, dropout_rate, pool_size)([visual_features, textual_features])

output = Dense(num_of_classes, activation="softmax")(mfh_output)

# Create model
model_mfh = Model(inputs=[base_model.input, sequence_input], outputs=output )

In [ ]:
# Compile the model
model_mfh.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [ ]:
# Train the model
keras.backend.clear_session()
model_mfh.fit([train_images, train_seq], train['class'],
              epochs=25, batch_size=64,
              validation_data=([valid_images, valid_seq], valid['class']),
              callbacks=callback_list)

In [ ]:
# Evaluate the model on the test set
model_mfh.load_weights(models_path + 'mfh_allclass.h5')
test_loss, test_accuracy = model_mfh.evaluate([test_images, test_seq], test['class'])
print(f'Test Loss: {round(test_loss,3)}, Test Accuracy: {round(test_accuracy,4)}')

25/25 [==============================] - 69s 3s/step - loss: 1.6655 - accuracy: 0.5995
Test Loss: 1.665, Test Accuracy: 0.5995


In [ ]:
# prediction
y_pred=model_mfh.predict([test_images, test_seq])
y_pred= np.argmax(y_pred, axis=-1)
## print matrices
print_metrices(test['class'],y_pred)

25/25 [==============================] - 69s 3s/step
Accuracy :  0.5995
Precison :  0.5612
Recall :  0.5995
F1 :  0.5695


In [ ]:
test['mfh'] = y_pred
test['mfh'] = test['mfh'].apply(label_map)
test

In [ ]:
yes_no = test[test['category']=='yes/no']
counting = test[test['category']=='counting']
other = test[test['category']=='other']

In [ ]:
# Generate the classification report
print_metrices(yes_no['answers'], yes_no['mfh'])

Accuracy :  0.8312
Precison :  0.9581
Recall :  0.8312
F1 :  0.8703


# Vanilla VQA

In [ ]:
# Clear Keras backend session
tf.keras.backend.clear_session()
# Build the VQA model
image_input_model_1 = Input(shape=(224, 224, 3))
mobilenet_model = MobileNetV2(include_top=False, weights='imagenet', input_tensor=image_input_model_1)
# Freeze the MobileNetV2 layers
for layer in mobilenet_model.layers:
     layer.trainable = False
# Process the image with ResNet
image_features_model_1 = tf.keras.layers.GlobalAveragePooling2D()(mobilenet_model.output)
cnn_layer_output = Dense(1024, activation='relu')(image_features_model_1)
cnn_model_1 = Model(inputs=image_input_model_1, outputs=cnn_layer_output)


# Process the question with an LSTM
question_input_model_1 = Input(shape=(15,))
question_embedding_model_1 = Embedding(input_dim=vocab_size, output_dim=300, input_length=10)(question_input_model_1)
lstm_features_1 = Bidirectional(LSTM(256, return_sequences=True))(question_embedding_model_1)
dropout_1 = tf.keras.layers.Dropout(0.5)(lstm_features_1)
lstm_features_2 = Bidirectional(LSTM(256,return_sequences=True))(dropout_1 )
dropout_2 = tf.keras.layers.Dropout(0.5)(lstm_features_2)
lstm_features_3 = Bidirectional(LSTM(256))(dropout_2)

#f1 = Flatten(name='flatten_1')(dropout2)
dense_layer_model_1 = Dense(1024, activation='relu')(lstm_features_3)
lstm_model_1 = Model(inputs=question_input_model_1, outputs=dense_layer_model_1)

# Combine image and text features
merged = concatenate([cnn_model_1.output, lstm_model_1.output], name='Fusion')
dropout_3 = tf.keras.layers.Dropout(0.5)(merged)
output = Dense(num_of_classes, activation='softmax')(dropout_3)

model = Model(inputs=[cnn_model_1.input, lstm_model_1.input], outputs=output)

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])


9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
# Train the model
keras.backend.clear_session()
history = model.fit([train_images, train_seq], train['class'],
          epochs=9,batch_size=64,
          validation_data=([valid_images, valid_seq], valid['class']),callbacks=callback_list)

In [ ]:
# Evaluate the model on the test set
model.load_weights(models_path+'vanilla_allclass.h5')
test_loss, test_accuracy = model.evaluate([test_images, test_seq], test['class'])
print(f'Test Loss: {round(test_loss,3)}, Test Accuracy: {round(test_accuracy,4)}')

25/25 [==============================] - 9s 290ms/step - loss: 1.7404 - accuracy: 0.6021
Test Loss: 1.74, Test Accuracy: 0.6021


In [ ]:
# prediction
y_pred=model.predict([test_images, test_seq])
y_pred= np.argmax(y_pred, axis=-1)
## print matrices
print_metrices(test['class'],y_pred)

25/25 [==============================] - 7s 247ms/step
Accuracy :  0.6021
Precison :  0.6111
Recall :  0.6021
F1 :  0.5783


In [ ]:
test['vanilla'] = y_pred
test['vanilla'] = test['vanilla'].apply(label_map)
test

In [ ]:
yes_no = test[test['category']=='yes/no']
counting = test[test['category']=='counting']
other = test[test['category']=='other']

In [ ]:
# Generate the classification report
print_metrices(counting['answers'], counting['vanilla'])

Accuracy :  0.5016
Precison :  0.574
Recall :  0.5016
F1 :  0.5005


# Top Down Attention (2022)

In [ ]:
#first attention
class First_Attention(tf.keras.Model):
      def __init__(self, units):
          super(First_Attention, self).__init__()
          self.W1 = tf.keras.layers.Dense(units)
          self.W2 = tf.keras.layers.Dense(units)
          self.V = tf.keras.layers.Dense(1)

      def call(self, features, hidden):
          # hidden shape == (batch_size, hidden size)
          # hidden_with_time_axis shape == (batch_size, 1, hidden size)
          # we are doing this to perform addition to calculate the score
          hidden_with_time_axis = tf.expand_dims(hidden, 1)

          # score shape == (batch_size, max_length, 1)
          # we get 1 at the last axis because we are applying score to self.V
          # the shape of the tensor before applying self.V is (batch_size, max_length, units)
          score = tf.nn.tanh(
              self.W1(features) + self.W2(hidden_with_time_axis))
          # attention_weights shape == (batch_size, max_length, 1)
          attention_weights = tf.nn.softmax(self.V(score), axis=1)

          # context_vector shape after sum == (batch_size, hidden_size)
          context_vector = (attention_weights * hidden_with_time_axis)+ features
          context_vector = tf.reduce_sum(context_vector, axis=1)
          return context_vector, attention_weights


In [ ]:
#Sceond attention
class Second_Attention(tf.keras.Model):
      def __init__(self, units):
          super(Second_Attention, self).__init__()
          self.W1 = tf.keras.layers.Dense(units)
          self.W2 = tf.keras.layers.Dense(units)
          self.V = tf.keras.layers.Dense(1)

      def call(self, features, hidden):
          # hidden shape == (batch_size, hidden size)
          # hidden_with_time_axis shape == (batch_size, 1, hidden size)
          # we are doing this to perform addition to calculate the score
          hidden_with_time_axis = tf.expand_dims(hidden, 1)
          feature_with_time_axis = tf.expand_dims(features, 1)

          # score shape == (batch_size, max_length, 1)
          # we get 1 at the last axis because we are applying score to self.V
          # the shape of the tensor before applying self.V is (batch_size, max_length, units)
          score = tf.nn.tanh(
              self.W1(feature_with_time_axis) + self.W2(hidden_with_time_axis))
          # attention_weights shape == (batch_size, max_length, 1)
          attention_weights = tf.nn.softmax(self.V(score), axis=1)

          # context_vector shape after sum == (batch_size, hidden_size)
          context_vector = (attention_weights * hidden_with_time_axis)+ feature_with_time_axis
          context_vector = tf.reduce_sum(context_vector, axis=1)
          return context_vector, attention_weights


**Define a model with attention (Vgg19+BiGRU+2 layers attention)**

In [ ]:
# Clear Keras backend session
tf.keras.backend.clear_session()
# Build the VQA model with VGG19
image_input_model_2 = Input(shape=(224, 224, 3))
vgg19_model = VGG19(weights='imagenet', include_top=False, input_tensor=image_input_model_2)
# Freeze the VGG19 layers
for layer in vgg19_model.layers:
    layer.trainable = False

# Process the image with VGG19
image_features_model_2 = Flatten()(vgg19_model.output)
vgg_output = Dense(100, activation='relu')(image_features_model_2 )


# Process the question with a GRU
question_input_model_2 = Input(shape=(15,))
question_embedding_model_2 = Embedding(input_dim=vocab_size, output_dim=300, input_length=10)(question_input_model_2)
gru_features = Bidirectional(GRU(50, return_sequences=True))(question_embedding_model_2)
# Add the VGG19 features and GRU
# First Attention
context_vector_1, attention_weights_1 = First_Attention(10)(gru_features,vgg_output)
# Second Attention
context_vector_2, attention_weights_2 = Second_Attention(10)(context_vector_1,vgg_output)
# Add the attention features
dense_layer_model_2 = Dense(64, activation='relu')(context_vector_2)
output_model_2 = Dense(num_of_classes, activation='softmax')(dense_layer_model_2 )
# Model
model_attention_2 = Model(inputs=[vgg19_model.input, question_input_model_2], outputs=output_model_2)

#Compile the model
model_attention_2.compile(loss='sparse_categorical_crossentropy', optimizer=RMSprop(learning_rate=0.001), metrics=['accuracy'])

80134624/80134624 [==============================] - 0s 0us/step


In [ ]:
# Train the model
keras.backend.clear_session()
model_attention_2.fit([train_images, train_seq], train['class'],
          epochs=30,batch_size=64,
          validation_data=([valid_images, valid_seq], valid['class']),callbacks=callback_list)

In [ ]:
# Evaluate the model on the test set
model_attention_2.load_weights(models_path+'tda_allclass.h5')
test_loss, test_accuracy = model_attention_2.evaluate([test_images, test_seq], test['class'])
print(f'Test Loss: {round(test_loss,3)}, Test Accuracy: {round(test_accuracy,4)}')

25/25 [==============================] - 96s 4s/step - loss: 2.5368 - accuracy: 0.4974
Test Loss: 2.537, Test Accuracy: 0.4974


In [ ]:
# prediction
y_pred=model_attention_2.predict([test_images, test_seq])
y_pred= np.argmax(y_pred, axis=-1)
## print matrices
print_metrices(test['class'],y_pred)

25/25 [==============================] - 91s 4s/step
Accuracy :  0.4974
Precison :  0.4756
Recall :  0.4974
F1 :  0.4706


In [ ]:
test['tda'] = y_pred
test['tda'] = test['tda'].apply(label_map)
test

In [ ]:
yes_no = test[test['category']=='yes/no']
counting = test[test['category']=='counting']
other = test[test['category']=='other']

In [ ]:
# Generate the classification report
print_metrices(other['answers'], other['tda'])

Accuracy :  0.5911
Precison :  0.5932
Recall :  0.5911
F1 :  0.5777
